# Compagnon Immobilier - Économie & Démographie

## Configuration de l'environnement

---

Projet fil rouge réalisé dans le cadre de la formation Data Scientist (DataScientest).

L'objectif du projet *Compagnon Immobilier* est de développer une solution d'aide à la décision pour les acheteurs immobiliers, en croisant des données sur les prix, la démographie, les transports, les services, etc.

Ce notebook correspond à la **partie Économie & Démographie** : nous y collectons et analysons les données économiques et démographiques issues de l'INSEE.

## 1. Périmètre de notre contribution

Dans le cadre du projet *Compagnon Immobilier*, nous nous concentrons sur les **indicateurs économiques et démographiques** susceptibles d'influencer les prix de l'immobilier et d'éclairer les décisions d'achat.

### 1.1 Indicateurs retenus

Après étude des sources disponibles, nous avons sélectionné les indicateurs suivants :

| Indicateur | Intérêt pour le projet |
|------------|------------------------|
| PIB trimestriel | Conjoncture économique nationale |
| Taux de chômage par département | Dynamisme économique local |
| Population et pyramide des âges | Attractivité démographique du territoire |
| Revenus des ménages | Pouvoir d'achat et capacité d'emprunt |
| Taux de pauvreté | Indicateur de précarité sociale |
| Indices des prix des logements | Variable cible pour la modélisation |
| Indice des prix à la consommation | Inflation et évolution du coût de la vie |

### 1.2 Organisation du travail

Notre partie se décompose en quatre notebooks :
1. **Configuration de l'environnement** (présent notebook)
2. **Extraction des données** via l'API INSEE
3. **Nettoyage et preprocessing**
4. **Analyse exploratoire et visualisations**

## 2. Configuration de l'environnement

### 2.1 Prérequis techniques

Le projet repose sur Python 3.x et les bibliothèques suivantes :
- **pandas** : manipulation de données tabulaires
- **numpy** : calculs numériques
- **matplotlib** / **seaborn** : visualisations graphiques
- **pynsee** : interface Python pour l'API INSEE

### 2.2 Installation des dépendances

Les bibliothèques nécessaires peuvent être installées via la commande ci-dessous :

In [12]:
# Kernel utilisé : "Python (Compagnon Immo)"
# 
# Si besoin de réinstaller manuellement (dans le terminal pas sur jupyter) :
# source venv/bin/activate
# pip install pandas numpy matplotlib seaborn pynsee



### 2.3 Imports

Nous importons ici l'ensemble des bibliothèques utilisées tout au long du projet :

In [13]:
# Bibliothèques standard
import os
import warnings
from datetime import datetime

# Manipulation de données
import pandas as pd
import numpy as np

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration des affichages
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes
pd.set_option('display.max_rows', 100)      # Limiter à 100 lignes
pd.set_option('display.width', None)        # Largeur automatique
warnings.filterwarnings('ignore')           # Masquer les avertissements

# Style des graphiques
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print(f"Pandas version : {pd.__version__}")
print(f"NumPy version : {np.__version__}")
print(f"Date d'exécution : {datetime.now().strftime('%Y-%m-%d %H:%M')}")

Pandas version : 2.3.3
NumPy version : 2.4.0
Date d'exécution : 2025-12-24 11:09


### 2.4 Import de pynsee

La bibliothèque **pynsee** constitue notre point d'accès principal aux données de l'INSEE. Elle permet d'interroger directement l'API sans avoir à gérer manuellement les requêtes HTTP.

Documentation officielle : https://pynsee.readthedocs.io/

In [3]:
# Import de pynsee
try:
    from pynsee.macrodata import get_dataset_list, get_series_list, get_series
    from pynsee.localdata import get_local_data
    print("pynsee importé avec succès")
except ImportError:
    print("Erreur : pynsee n'est pas installé")
    print("Exécuter : pip install pynsee")

pynsee importé avec succès


## 3. Structure du projet

### 3.1 Arborescence des fichiers

Nous définissons ici les chemins d'accès aux différents répertoires du projet :

In [ ]:
# Définition des chemins
PROJECT_ROOT = os.path.dirname(os.getcwd())  # Racine du projet
DATA_DIR = os.path.join(PROJECT_ROOT, 'Economie_Demographie', 'data')

print(f"Racine du projet : {PROJECT_ROOT}")
print(f"Dossier données : {DATA_DIR}")
print()

# Vérification de l'existence du dossier
if os.path.exists(DATA_DIR):
    print("Fichiers de données disponibles :")
    for f in sorted(os.listdir(DATA_DIR)):
        if f.endswith('.csv'):
            size = os.path.getsize(os.path.join(DATA_DIR, f)) / 1024
            print(f"  - {f} ({size:.1f} KB)")
else:
    print("Le dossier de données n'existe pas encore")

### 3.2 Présentation des sources de données

Le tableau ci-dessous recense les fichiers de données utilisés dans ce projet, ainsi que leurs caractéristiques principales :

| Fichier | Description | Source INSEE | Période |
|---------|-------------|--------------|----------|
| pib_france.csv | Taux de croissance du PIB trimestriel | Séries IDBANK | 1949-2025 |
| chomage_departements_complet.csv | Taux de chômage par département | TAUX-CHOMAGE | 1982-2025 |
| population_departements_series.csv | Population par département et tranche d'âge | TCRED-ESTIMATIONS-POPULATION | 1975-2025 |
| revenus_departements.csv | Revenus et indicateurs de pauvreté | FILOSOFI 2018 | 2018 |
| emploi_departements.csv | Emploi salarié par secteur d'activité | Flores 2018 | 2018 |
| entreprises_departements.csv | Stock et créations d'entreprises | REE 2019-2020 | 2019-2020 |
| ipc_france.csv | Indice des prix à la consommation | Séries IDBANK | 1990-2025 |
| prix_logements_indices.csv | Indices des prix des logements | Notaires-INSEE | 1992-2025 |

## 4. Vérification de la connexion à l'API INSEE

Avant de procéder à l'extraction des données, nous vérifions que la connexion à l'API fonctionne correctement :

In [ ]:
# Test de connexion à l'API INSEE
try:
    # Récupération de la liste des datasets disponibles
    datasets = get_dataset_list()
    print(f"Connexion à l'API INSEE : OK")
    print(f"Nombre de datasets disponibles : {len(datasets)}")
    print()
    
    # Affichage de quelques datasets
    print("Exemples de datasets :")
    for i, row in datasets.head(5).iterrows():
        print(f"  - {row['id']} : {row['Name.fr'][:60]}...")
        
except Exception as e:
    print(f"Erreur de connexion : {e}")
    print("Vérifier la connexion internet ou l'installation de pynsee")

## 5. Fonction utilitaire de chargement

Fonction pour charger les fichiers CSV avec gestion des erreurs :

In [ ]:
def charger_donnees(nom_fichier, afficher_info=True):
    """
    Charge un fichier CSV depuis le dossier de données.
    
    Paramètres:
        nom_fichier (str): Nom du fichier CSV à charger
        afficher_info (bool): Afficher les informations sur le DataFrame
    
    Retourne:
        DataFrame pandas ou None en cas d'erreur
    """
    chemin = os.path.join(DATA_DIR, nom_fichier)
    
    try:
        df = pd.read_csv(chemin)
        
        if afficher_info:
            print(f"Fichier chargé : {nom_fichier}")
            print(f"  Dimensions : {df.shape[0]} lignes x {df.shape[1]} colonnes")
            print(f"  Colonnes : {list(df.columns)[:5]}..." if len(df.columns) > 5 else f"  Colonnes : {list(df.columns)}")
            print()
        
        return df
    
    except FileNotFoundError:
        print(f"Fichier non trouvé : {chemin}")
        return None
    except Exception as e:
        print(f"Erreur lors du chargement : {e}")
        return None

### Test de la fonction de chargement

In [14]:
# Test de chargement d'un fichier
df_test = charger_donnees('pib_france.csv')

if df_test is not None:
    print("Aperçu des données :")
    display(df_test.head(3))

Fichier chargé : pib_france.csv
  Dimensions : 306 lignes x 56 colonnes
  Colonnes : ['TIME_PERIOD', 'OBS_VALUE', 'OBS_STATUS', 'OBS_QUAL', 'OBS_TYPE']...

Aperçu des données :


,TIME_PERIOD,OBS_VALUE,OBS_STATUS,OBS_QUAL,OBS_TYPE,IDBANK,FREQ,TITLE_FR,TITLE_EN,LAST_UPDATE,UNIT_MEASURE,UNIT_MULT,REF_AREA,DECIMALS,OBS_REV,DATE,nomflow,cleFlow,INDICATEUR,SECT-INST,OPERATION,CNA_PRODUIT,NATURE,VALORISATION,CORRECTION,SERIE_ARRETEE,FREQ_label_fr,FREQ_label_en,INDICATEUR_label_fr,INDICATEUR_label_en,SECT-INST_label_fr,SECT-INST_label_en,OPERATION_label_fr,OPERATION_label_en,CNA_PRODUIT_label_fr,CNA_PRODUIT_label_en,NATURE_label_fr,NATURE_label_en,REF_AREA_label_fr,REF_AREA_label_en,VALORISATION_label_fr,VALORISATION_label_en,UNIT_MEASURE_label_fr,UNIT_MEASURE_label_en,CORRECTION_label_fr,CORRECTION_label_en,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en,OBS_STATUS_label_fr,OBS_STATUS_label_en,OBS_REV_label_fr,OBS_REV_label_en,OBS_TYPE_label_fr,OBS_TYPE_label_en,OBS_QUAL_label_fr,OBS_QUAL_label_en
0,1949-Q2,0.9,A,DEF,A,11794844,T,Comptes trimestriels (base 2020) - Évolution d...,Quarterly accounts (base 2020) - Evolution of ...,2025-11-28,POURCENT,0,FE,1,NaN,1949-04-01 00:00:00,CNT-2020-PIB-EQB-RF,T.CNT-EQUILIBRE_PIB.SO.PIB.SO.TAUX.FE.L.POURCE...,CNT-EQUILIBRE_PIB,SO,PIB,SO,TAUX,L,CVS-CJO,False,Trimestrielle,Quarterly,Équilibre du produit intérieur brut,Gross Domestic Product balance,Sans objet,Not applicable,PIB - Produit intérieur brut,GDP - Gross domestic product,Sans objet,Not applicable,Taux,Rate,France,France,Volumes aux prix de l'année précédente chaînés,Volumes chained at previous year prices,%,%,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
1,1949-Q3,1.5,A,DEF,A,11794844,T,Comptes trimestriels (base 2020) - Évolution d...,Quarterly accounts (base 2020) - Evolution of ...,2025-11-28,POURCENT,0,FE,1,NaN,1949-07-01 00:00:00,CNT-2020-PIB-EQB-RF,T.CNT-EQUILIBRE_PIB.SO.PIB.SO.TAUX.FE.L.POURCE...,CNT-EQUILIBRE_PIB,SO,PIB,SO,TAUX,L,CVS-CJO,False,Trimestrielle,Quarterly,Équilibre du produit intérieur brut,Gross Domestic Product balance,Sans objet,Not applicable,PIB - Produit intérieur brut,GDP - Gross domestic product,Sans objet,Not applicable,Taux,Rate,France,France,Volumes aux prix de l'année précédente chaînés,Volumes chained at previous year prices,%,%,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
2,1949-Q4,1.4,A,DEF,A,11794844,T,Comptes trimestriels (base 2020) - Évolution d...,Quarterly accounts (base 2020) - Evolution of ...,2025-11-28,POURCENT,0,FE,1,NaN,1949-10-01 00:00:00,CNT-2020-PIB-EQB-RF,T.CNT-EQUILIBRE_PIB.SO.PIB.SO.TAUX.FE.L.POURCE...,CNT-EQUILIBRE_PIB,SO,PIB,SO,TAUX,L,CVS-CJO,False,Trimestrielle,Quarterly,Équilibre du produit intérieur brut,Gross Domestic Product balance,Sans objet,Not applicable,PIB - Produit intérieur brut,GDP - Gross domestic product,Sans objet,Not applicable,Taux,Rate,France,France,Volumes aux prix de l'année précédente chaînés,Volumes chained at previous year prices,%,%,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value


---

## Bilan

L'environnement de travail est désormais opérationnel. Nous avons vérifié que :

- Les bibliothèques Python nécessaires (pandas, numpy, matplotlib, seaborn) sont correctement installées
- La bibliothèque pynsee permet d'accéder aux données de l'INSEE
- La connexion à l'API est fonctionnelle
- Les fichiers de données existants sont accessibles

Nous pouvons à présent passer à l'étape suivante : l'extraction des données économiques et démographiques via l'API INSEE.